## Work-In-Progress ... Demo Notebook

In [1]:
from ipywidgets import widgets
from IPython.display import Javascript, HTML
import numpy as np
import pymagicc

from bokeh.io import push_notebook, show, output_notebook
from bokeh.plotting import figure
from bokeh.layouts import row
output_notebook(hide_banner=True);

from functools import lru_cache

In [2]:
scen = pymagicc.scenarios["RCP3PD"]
data = scen["WORLD"]["FossilCO2"].loc[:2100]
emissions = figure(
    title="Scenario", 
    plot_height=500, 
    plot_width=700,
    y_range=(-3, 30),
    sizing_mode="scale_width"
)
r = emissions.line(data.index, data.values, color="blue", line_width=3)

output = pymagicc.run(scen)
temp = output["SURFACE_TEMP"]["GLOBAL"]
temperature = figure(
    title="Global Surface Temperature", 
    plot_height=500, 
    plot_width=700,
    y_range=(0,8),
    sizing_mode="scale_width"
)
t = temperature.line([0], [0], color="red", line_width=3)


In [3]:
set_working = Javascript("document.getElementsByClassName('bk-plot-layout')[1].setAttribute('style', 'opacity: 0.1');")
set_done = Javascript("document.getElementsByClassName('bk-plot-layout')[1].setAttribute('style', 'opacity: 1');")

In [4]:
@lru_cache(maxsize=32)
def run_magicc(scenario, core_climatesensitivity):
    return pymagicc.run(pymagicc.scenarios[scenario], core_climatesensitivity=core_climatesensitivity)

In [5]:
def update(scenario, gas, core_climatesensitivity, output_variable, start, end):
    if end < start:
        reference_period_end.value = start
    display(set_working)
    scen = pymagicc.scenarios[scenario]
    data = scen["WORLD"][gas].loc[:2100]
    r.data_source.data['x'] = data.index
    r.data_source.data['y'] = data.values
    push_notebook()  
    output = run_magicc(scenario, core_climatesensitivity)
    out = output[output_variable]["GLOBAL"]
    if output_variable == "CO2_CONC":
        temperature.title.text = "CO₂  Concentration "
        temperature.y_range.start = 0
        temperature.y_range.end = 1000
    elif output_variable == "SURFACE_TEMP":
        
        temperature.y_range.start = -.5
        temperature.y_range.end = 6
        if start == end:
            out = out - out.loc[start]
            temperature.title.text = "Global Surface Temperature (rel. to {})".format(start)
        else:
            temperature.title.text = "Global Surface Temperature (rel. to mean {}-{})".format(start, end)
            out = out - out.loc[start:end].mean()
        out = out.loc[1850:]
    elif output_variable == "GHG_RF":
        temperature.title.text = "Radiative Forcing (GHG)"
        temperature.y_range.start = 0
        temperature.y_range.end = 10
    
    t.data_source.data = {'x': out.index, 'y': out.values}
    
    push_notebook()    
    display(set_done)

In [6]:
handle = show(row(emissions, temperature), notebook_handle=True)

scenario = widgets.Dropdown(
    options=["RCP3PD", "RCP45", "RCP6", "RCP85"],
    description="Scenario"
)
output_variable = widgets.Dropdown(
    options=[
        ("Global Surface Temperature", "SURFACE_TEMP"),
        ("CO₂ Concentration", "CO2_CONC"),
        ("Radiative Forcing (GHG)", "GHG_RF")
    ],
    description="Output Variable"
)
gas = widgets.Dropdown(options=["FossilCO2", "OtherCO2", "CH4", "N2O"], description="Greenhouse Gas")
core_climatesensitivity = widgets.FloatSlider(
    value=3, min=1.5, max=4.5, step=0.1, continuous_update=False,
    description="Climate Sensitivity"
)

reference_period_start = widgets.BoundedIntText(
    description="Start Temperature Reference Period",
    value=1850,
    min=1850,
    max=2015,
    step=1,
    continuous_update=False
)

reference_period_end = widgets.BoundedIntText(
    description="End Temperature Reference Period",
    value=1900,
    min=1850,
    max=2015,
    step=1,
    continuous_update=False
)

ui = widgets.VBox([ 
        widgets.HBox([
            widgets.VBox([scenario, core_climatesensitivity]),
            widgets.VBox([gas, output_variable])
        ]),
        widgets.HBox(
            [reference_period_start, reference_period_end])
])

out = widgets.interactive_output(
    update, 
    {
        'scenario': scenario, 
        'gas': gas,
        'core_climatesensitivity': core_climatesensitivity,
        "output_variable": output_variable,
        "start": reference_period_start,
        "end": reference_period_end
    }
)
display(ui, out)
display(HTML("<style>.container { width:90% !important; }</style>"))

VBox(children=(HBox(children=(VBox(children=(Dropdown(description='Scenario', options=('RCP3PD', 'RCP45', 'RCP6', 'RCP85'), value='RCP3PD'), FloatSlider(value=3.0, continuous_update=False, description='Climate Sensitivity', max=4.5, min=1.5))), VBox(children=(Dropdown(description='Greenhouse Gas', options=('FossilCO2', 'OtherCO2', 'CH4', 'N2O'), value='FossilCO2'), Dropdown(description='Output Variable', options=(('Global Surface Temperature', 'SURFACE_TEMP'), ('CO₂ Concentration', 'CO2_CONC'), ('Radiative Forcing (GHG)', 'GHG_RF')), value='SURFACE_TEMP'))))), HBox(children=(BoundedIntText(value=1850, description='Start Temperature Reference Period', max=2015, min=1850), BoundedIntText(value=1900, description='End Temperature Reference Period', max=2015, min=1850)))))

Output()